In [6]:
import matplotlib
import math
from pylab import *
import numpy as np
import itertools
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import os
import imageio
import random
import statistics as st
import scipy.stats as ss
import matplotlib.patches as patches


#Parámetros
N = [10,20,30,40,50,60,70,80,90] #valores de n para el experimento
pt = 1 #paso temporal
s = 100 #cantidad de simulaciones por densidad
t = 500 #tiempo que dura cada simulación
vemax = 1 #máxima velocidad que puede tener un agente
vemin= 0 #mínima velocidad
tM= 50 #capacidad máxima de agentes por carril

class Vehiculo:
  id_iter = itertools.count() #para crearles un id a los agentes
  def __init__(self,tipo,posx,velocidad,posy,eje,estado,ejeinicial):
      self.id = next(Vehiculo.id_iter) #id
      self.t = tipo #para poder duplicar agentes y simular el espacio toroidal
      self.x = posx #para la posición del vehículo, coordenada horizontal
      self.y = posy #para la posición del vehículo, coordenada vertical
      self.vm = velocidad #velocidad del vehículo en el paso
      self.d = 0 #distancia recorrida por paso
      self.a = 0.01 #valor default de la aceleración
      self.m = random.choice(["vm","vemax"]) #etiqueta del valor considerado para comportamiento en la distancia de seguimiento, para este experimento es aleatorio
      self.f = vemax #valor default considerado para comportamiento en la distancia de seguimiento.
      self.j = eje #eje por el cual se mueve el agente (X o Y)
      self.e = estado #libre o parar para hacer que un agente no se mueva si está el semáforo en rojo.
      self.se = "Si" #etiqueta para identificar a los que no respetan la luz roja
      self.jo=ejeinicial #etiqueta para pintar al carrito según su eje inicial
      self.g="No" #para identificar a los agentes que giran al otro carril y evitar doble giro
      self.fr="No" #para identificar a los agentes que están por el semáforo y son transgresores
      self.im="No" #para identificar a los transgresores
      self.com="No" #para identificar a los oportunistas que están por el semáforo
      self.compor="Responsable" #valor default de los agentes, se puede variar a indiferente, oportunista y transgresor

  def mover(self,acel): #función para la actualización de la distancia recorrida por un agente
      self.a = acel
      self.vm = max(vemin, min(vemax, self.vm + self.a*pt))
      self.d =self.vm*pt


class Carril: #clase Carriles
    def __init__(self,pos,eje,ncarril):
        self.p = pos #donde se ubica
        self.j = eje #si es eje X o Y

class Semaf:
    def __init__(self,posx,posy,eje,semaforo):
        self.x = posx #para la posición del semáforo, coordenada horizontal
        self.y = posy #para la posición del semáforo, coordenada vertical
        self.j = eje #para qué carril es
        self.s = semaforo #off es verde, on es rojo

def iniciar(n): #con esta función se crean los agentes del modelo
  global vehiculos, filenames, carriles,semafs,ti,te,refx,refy
  filenames = [] #lista para guardar las imágenes del GIF de la simulación
  vehiculos = [] #lista para guardar todos los agentes tipo vehículo.

  semafs=[] #lista de semáforos
  carriles = [] #lista de carriles
  ti=0 #para el tiempo en el semáforo
  refx=0 #para que un transgresor pueda frenar sin sentido en la luz verde cada un paso.
  refy=0

  s1 = Semaf(29.75,20.75,"ejex","off") #semáforo 1
  semafs.append(s1)
  s2 = Semaf(30.7,19.5,"ejey","off") #semáforo 2
  semafs.append(s2)

  l1 = Carril(20,"ejex",1) #carril 1
  carriles.append(l1)
  l6 = Carril(30.5,"ejey",2) #carril 2
  carriles.append(l6)


  #para crear agentes con comportamiento transgresor en cada carril
  while len(vehiculos) < n*0.125:
    v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),20,"ejex","libre","ejex")
    if round(v.x,0) not in [round(z.x,0) for z in vehiculos if v.y == z.y and z.j=="ejex"]:
      v.im = "Si"
      v.compor="Transgresores"
      vehiculos.append(v)

  while len(vehiculos) < n*0.25:
    v = Vehiculo("carro",30.5, round(np.random.uniform(0.1,0.3),4),round(np.random.uniform(0,tM),0), "ejey","libre","ejey")
    if round(v.y,0) not in [round(z.y,0) for z in vehiculos if v.x == z.x and z.j=="ejey"]:
      v.im = "Si"
      v.compor="Transgresores"
      vehiculos.append(v)

  #para crear agentes con comportamiento indiferente en cada carril
  while len(vehiculos) < n*0.375:
    v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),20,"ejex","libre","ejex")
    if round(v.x,0) not in [round(z.x,0) for z in vehiculos if v.y == z.y and z.j=="ejex" ]:
      v.se = "No"
      v.compor="Indiferentes"
      vehiculos.append(v)

  while len(vehiculos) < n*0.5:
    v = Vehiculo("carro",30.5, round(np.random.uniform(0.1,0.3),4),round(np.random.uniform(0,tM),0), "ejey","libre","ejey")
    if round(v.y,0) not in [round(z.y,0) for z in vehiculos if v.x == z.x and z.j=="ejey"]:
      v.se = "No"
      v.compor="Indiferentes"
      vehiculos.append(v)

  #para crear agentes con comportamiento oportunista en cada carril
  while len(vehiculos) < n*0.625:
    v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),20,"ejex","libre","ejex")
    if round(v.x,0) not in [round(z.x,0) for z in vehiculos if v.y == z.y and z.j=="ejex"]:
      v.se = "No"
      v.compor="Oportunistas"
      vehiculos.append(v)

  while len(vehiculos) < n*0.75:
    v = Vehiculo("carro",30.5, round(np.random.uniform(0.1,0.3),4),round(np.random.uniform(0,tM),0), "ejey","libre","ejey")
    if round(v.y,0) not in [round(z.y,0) for z in vehiculos if v.x == z.x and z.j=="ejey"]:
      v.se = "No"
      v.compor="Oportunistas"
      vehiculos.append(v)

  #para crear agentes con comportamiento responsable en cada carril
  while len(vehiculos) < n*0.875:
    v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),20,"ejex","libre","ejex")
    if round(v.x,0) not in [round(z.x,0) for z in vehiculos if v.y == z.y and z.j=="ejex"]:
      v.compor="Responsables"
      vehiculos.append(v)

  while len(vehiculos) < n:
    v = Vehiculo("carro",30.5, round(np.random.uniform(0.1,0.3),4),round(np.random.uniform(0,tM),0), "ejey","libre","ejey")
    if round(v.y,0) not in [round(z.y,0) for z in vehiculos if v.x == z.x and z.j=="ejey"]:
      v.compor="Responsables"
      vehiculos.append(v)


def toroide(): #para simular un espacio toroidal
  for v in [v for v in vehiculos if v.j == "ejex"]:
    if v.x > tM:
        v.x = v.x % tM

  for v in [v for v in vehiculos if v.j == "ejey"]:
    if v.y > tM:
        v.y = v.y % tM

  for v in vehiculos: #se resetean estos atributos al valor default en cada paso
    v.g="No"
    v.e="libre"
    v.com="No"


def duplicarx(): #función para duplicar al agente que está al inicio del carril y ubicarlo adelante del que está al último para que el último sepa cómo moverse en eje X
  for c in carriles:
    if len([v for v in vehiculos if v.y == c.p and v.j == "ejex"])!= 0:
      for v in vehiculos:
        if v.x == min([v.x for v in vehiculos if v.j == "ejex" and v.y == c.p]):
          ax = v.vm
          copiax = Vehiculo("doble", min([v.x for v in vehiculos if v.j == "ejex" and v.y == c.p])+tM, ax, c.p,"ejex","libre","ejex")
          vehiculos.append(copiax)


def duplicary(): #función para duplicar al agente que está al inicio del carril y ubicarlo adelante del que está al último para que el último sepa cómo moverse en eje Y
  for c in carriles:
    if len([v for v in vehiculos if v.x == c.p and v.j == "ejey"])!= 0:
      for v in vehiculos:
        if v.y == min([v.y for v in vehiculos if v.j == "ejey" and v.x == c.p]):
          ay = v.vm
          copiay = Vehiculo("doble", c.p, ay, min([v.y for v in vehiculos if v.j == "ejey" and v.x == c.p])+tM, "ejey","libre","ejey")
          vehiculos.append(copiay)


def parametros(v,vm): #función para que cada carro actualice su parámetro de comportamiento en base a la etiqueta que se le asignó en la función iniciar()
  distancias = {
      ("vemax"): vemax,
      ("vm"): vm}
  return distancias.get((v),"NA")

def movimientox(t): #función para que cada agente tipo vehículo sepa cómo moverse en el eje X
  global refx
  for c in carriles:
    vehiculos.sort(key = lambda v: v.x)
    for v,z in zip([v for v in vehiculos if v.j == "ejex" and v.y==c.p],[z for z in vehiculos if z.j == "ejex" and z.y==c.p][1:]):
      d = z.x - v.x #distancia entre vehículos
      if v.e == "libre": #solo pueden acelerar o desacelerar los que tienen este estado
        v.f = parametros(v.m,(min(1,max(v.vm + 0.011,0.1)))) #actualización del valor considerado para el comportamiento en la distancia de seguimiento.
        if len([w for w in vehiculos  if 19.6<=w.y<=20.1  and w.j=="ejey"])!=0 and 29.5<v.x<30.5 and v.com=="Si": #aca los oportunistas consideran un mayor rango de cruce cuando la luz es roja, si no tienen ese espacio, frenan
          v.vm = 0
          v.mover(0)
        elif len([w for w in vehiculos if 0.5<w.x-v.x<1 and abs(w.y-v.y)<0.25 and w.j=="ejey"])!=0: #los indiferentes y transgresores cruzan en rojo solo evitando el choque
          v.vm=0
          v.mover(0)
        elif v.fr=="Si" and random.random()<0.5 and 1<t-refx: #transgresores paran sin sentido con un 50% de probabilidad en la luz verde
          v.vm = 0
          v.mover(0)
          refx=t
        elif d >= v.f*pt: #si está en verde o si hay espacio con el de adelante y su comportamiento no le hace frenar por los que cruzan, acelera.
          v.mover(0.01+0.001*random.uniform(-1,1))
        else: #o desacelera
          v.vm = z.vm #iguala la velocidad del vehículo de adelante
          v.mover(-0.02) #  y luego reduce su velocidad.
      elif v.e=="parar": #los que tienen este estado deben estar parados
        v.vm=0
        v.mover(0)

      v.x = v.d + v.x #se actualiza la nueva posición del vehículo en base a la distancia recorrida.


def movimientoy(t): #función para que cada agente tipo vehículo sepa cómo moverse en el eje Y (ver comentarios de función movimientoX).
  global refy
  for c in carriles:
    vehiculos.sort(key = lambda v: v.y)
    for v,z in zip([v for v in vehiculos if v.j == "ejey" and v.x==c.p],[z for z in vehiculos if z.j == "ejey" and z.x==c.p][1:]):
      d = z.y - v.y
      if v.e == "libre":
        v.f = parametros(v.m,(min(1,max(v.vm + 0.011,0.1))))
        if len([w for w in vehiculos if 30.1<=w.x<=30.6 and w.j=="ejex"])!=0 and 19<v.y<20 and v.com=="Si":
          v.vm=0
          v.mover(0)
        elif len([w for w in vehiculos if abs(w.x-v.x)<0.25 and 0<w.y-v.y<=0.5 and w.j=="ejex"])!=0:
          v.vm=0
          v.mover(0)
        elif v.fr=="Si" and random.random()<0.5 and 1<t-refy:
          v.vm = 0
          v.mover(0)
          refy=t
        elif d >= v.f*pt:
          v.mover(0.01+0.001*random.uniform(-1,1))
        else:
          v.vm = z.vm
          v.mover(-0.02)
      elif v.e=="parar":
        v.vm=0
        v.mover(0)

      v.y = v.d + v.y

def semaforo(t,pe): #el semáforo tiene una duración (pe=150)
  global ti
  for c in carriles:
    for s,s2 in zip([s for s in semafs if s.j == "ejex"],[s2 for s2 in semafs if s2.j == "ejey"]):
      if s.s == "off" and t == ti+(pe*2) or t == 1: #para hacer el cambio de luz cada pe
        s.s="on"
        s2.s ="off"
        ti=t
      elif t == ti+pe and s.s == "on":
        s.s="off"
        s2.s="on"

      if len([a for a in vehiculos if a.j== "ejey"])==0 and s.s=="on": #si un carril está vacío
        for b in [b for b in vehiculos if b.j== "ejex"]:
          if 29.5<=b.x<30 and b.compor=="Responsables": #siempre paran en luz roja
            b.e = "parar"
            b.vm = 0
          elif 29.5<=b.x<30 and b.compor=="Transgresores": #paran con un 50% de probabilidad
            if random.random()<0.5:
              b.e = "parar"
              b.vm = 0
          elif 29.5<=b.x<30 and b.compor=="Oportunistas":
            b.com="Si"
          else:
            pass


      elif len([b for b in vehiculos if b.j== "ejex"])==0 and s2.s=="on": #si el otro lo está
        for a in [a for a in vehiculos if a.j== "ejey"]:
          if 19<=a.y<19.5 and a.compor=="Responsables":
            a.e = "parar"
            a.vm = 0
          elif 19<=a.y<19.5 and a.compor=="Transgresores":
            if random.random()<0.5:
              a.e = "parar"
              a.vm = 0
          elif 19<=a.y<19.5 and a.compor=="Oportunistas":
            a.com="Si"
          else:
            pass

      else: #cuando ambos carriles tienen carros
        for b in [b for b in vehiculos if b.j== "ejex"]:
          for a in [a for a in vehiculos if a.j== "ejey"]:
            if s.s=="on":
              if a.im=="Si" and 19<=a.y<19.5: #los del verde, avanzan
                a.fr="Si"
              a.e="libre"
              if 29.5<=b.x<30: #los del rojo, se detienen si son responsables siempre.
                if b.compor=="Responsables":
                  b.e = "parar"
                  b.vm = 0
                elif b.compor=="Transgresores" and random.random()<0.5:
                  b.e = "parar"
                  b.vm = 0
                elif b.compor=="Oportunistas":
                  b.com="Si"
                else:
                  pass

            elif s2.s=="on":
              if b.im=="Si" and 29.5<=b.x<30:
                b.fr="Si"
              b.e="libre"
              if 19<=a.y<19.5:
                if a.compor=="Responsables":
                  a.e = "parar"
                  a.vm = 0
                elif a.compor=="Transgresores" and random.random()<0.5:
                  a.e = "parar"
                  a.vm = 0
                elif a.compor=="Oportunistas":
                  a.com="Si"
                else:
                  pass

def borrar(): #función para borrar el duplicado creado
  for c in carriles:
    for v in vehiculos:
      if v.t == "doble":
          vehiculos.remove(v)


def giro(): #función para que un agente pueda girar al otro carril al llegar al cruce con un 50% de probabilidad
  for c in carriles:
    for v in vehiculos:
      if v.j != c.j:
        if abs(v.x - c.p) <= 0.1 and v.g=="No" and random.random() < 0.5 and v.vm!=0:
          v.j=c.j
          v.x=c.p
          v.g="Si"
        elif abs(v.y - c.p) <= 0.1 and v.g=="No" and random.random() < 0.5 and v.vm!=0:
          v.j=c.j
          v.y=c.p
          v.g="Si"
      else:
        pass

def interaccion(n,s): #función que une las funciones que se necesitan en cada paso de la simulación
    global filenames
    for i in range(1,t+1): #se ejecuta el loop la cantidad de pasos (duración) que tiene cada simulación
        semaforo(i,150)
        giro()
        duplicarx()
        duplicary()
        movimientox(i)
        movimientoy(i)
        borrar()
        toroide()


        #Información recopilada por simulación para generar gráficos. Se puede recopilar otro tipo de información en base a lo requerido.
        vel1.append([n,st.mean([v.vm for v in vehiculos]),s])
        med1.append([n,st.median([v.vm for v in vehiculos]),s])
        des1.append([n,st.stdev([v.vm for v in vehiculos]),s])
        err1.append([n,ss.sem([v.vm for v in vehiculos]),s])

        velre.append([n,st.mean([v.vm for v in vehiculos if v.compor=="Responsables"]),s])
        velop.append([n,st.mean([v.vm for v in vehiculos if v.compor=="Oportunistas"]),s])
        velin.append([n,st.mean([v.vm for v in vehiculos if v.compor=="Indiferentes"]),s])
        veltr.append([n,st.mean([v.vm for v in vehiculos if v.compor=="Transgresores"]),s])

        #Código para generar una imagen de cada paso por simulación y poder generar el GIF de toda la simulación. Para hacer el GIF, se descomenta lo que viene a continuación:
        '''
        ig, ax = plt.subplots(figsize=(17,17))
        #carros
        plot([v.x for v in vehiculos if v.compor=="Oportunistas"], [v.y for v in vehiculos if v.compor=="Oportunistas"] ,'o', color="blue", markersize=3)
        plot([v.x for v in vehiculos if v.compor=="Indiferentes"], [v.y for v in vehiculos if v.compor=="Indiferentes"] ,'o', color="red", markersize=3)
        plot([v.x for v in vehiculos if v.compor=="Responsables"], [v.y for v in vehiculos if v.compor=="Responsables"] ,'o', color="green", markersize=3)
        plot([v.x for v in vehiculos if v.compor=="Transgresores"], [v.y for v in vehiculos if v.compor=="Transgresores"] ,'o', color="black", markersize=3)
        #semaforos de carros
        plot([s.x for s in semafs if s.s== "off"], [s.y for s in semafs if s.s== "off"] ,'gs', markersize=5)
        plot([s.x for s in semafs if s.s== "on"], [s.y for s in semafs if s.s== "on"] ,'rs', markersize=5)
        #semaforos peatonales
        #plot([s.x for s in semafs if s.s== "off" and s.t=="peatonal"], [s.y for s in semafs if s.s== "off" and s.t=="peatonal"] ,'gs', markersize=5)
        #plot([s.x for s in semafs if s.s== "on" and s.t=="peatonal"], [s.y for s in semafs if s.s== "on" and s.t=="peatonal"] ,'rs', markersize=5)
        #plot([s.x for s in semafs if s.p== "Si"], [s.y for s in semafs if s.p== "Si"] ,'ks', markersize=5)
        #carriles
        #ax.add_patch(patches.Rectangle((0, 20.25),50,0.5,edgecolor = 'white',facecolor = 'gray',fill=True))
        ax.add_patch(patches.Rectangle((0, 19.75),50,0.5,edgecolor = 'white',facecolor = 'gray',fill=True))
        #ax.add_patch(patches.Rectangle((29.75, 0),0.5,50,edgecolor = 'white',facecolor = 'gray',fill=True))
        ax.add_patch(patches.Rectangle((30.25, 0),0.5,50,edgecolor = 'white',facecolor = 'gray',fill=True))
        text(5,35,"Paso:")
        text(10,35,i)
        text(5,30,(n, "carros"))
        #text(5,28,("Periodo: ",pe))
        tick_params(labelcolor='w')
        #grid(b=True, axis='both', color = "white")
        axis([0,tM,0,tM])

        filename = f'paso{i}.png'
        filenames.append(filename)
        plt.savefig(filename)
        plt.close()
        #'''

        for v in vehiculos: #esto está acá para que se pueda pintar en el GIF a los transgresores que frenan en el verde sin sentido.
          v.fr="No"



In [ ]:
#Este bloque ejecuta el modelo
#listas para guardar la información de los gráficos
vel1 = []
med1 = []
des1 = []
err1 = []
velre=[]
velop=[]
veltr=[]
velin=[]

for i in N: #para cada cantidad de carros (i) por simulación, se ejecuta (j) cantidad de simulaciones.
  for j in range (1,s+1): #el s+1 es la cantidad de simulaciones
    iniciar(i)
    interaccion(i,j)
  mec=[]
  mere=[]
  meop=[]
  mein=[]
  metr=[]
  mec.append(st.mean([l[1] for l in vel1 if l[0]==i]))
  mere.append(st.mean([l[1] for l in velre if l[0]==i]))
  meop.append(st.mean([l[1] for l in velop if l[0]==i]))
  metr.append(st.mean([l[1] for l in veltr if l[0]==i]))
  mein.append(st.mean([l[1] for l in velin if l[0]==i]))

  print([mec,i],[mere,i],[meop,i],[metr,i],[mein,i])
  print("-----------------------------","Acabó N =",i,"-----------------------------")


In [ ]:
#Código para generar el GIF, siempre y cuando se hayan guardado las imágenes por paso
gif_name= "Los 4 comportamientos"

with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        i = imageio.imread(filename)
        writer.append_data(i)

#Esto elimina automáticamente las imágenes por paso creadas
for filename in set(filenames):
    os.remove(filename)